In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -d -p tensorflow,numpy

Sebastian Raschka 2017-08-19 

tensorflow 1.3.0
numpy 1.12.1


In [1]:
import struct
import numpy as np
import os


def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte'
                                % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte'
                               % kind)

    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII",
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)

    return images, labels


X_data, y_data = load_mnist('./', kind='train')
print('Rows: {},  Columns: {}'.format(X_data.shape[0], X_data.shape[1]))
X_test, y_test = load_mnist('./', kind='t10k')
print('Rows: {},  Columns: {}'.format(X_test.shape[0], X_test.shape[1]))

X_train, y_train = X_data[:50000,:], y_data[:50000]
X_valid, y_valid = X_data[50000:,:], y_data[50000:]

print('Training:   ', X_train.shape, y_train.shape)
print('Validation: ', X_valid.shape, y_valid.shape)
print('Test Set:   ', X_test.shape, y_test.shape)

Rows: 60000,  Columns: 784
Rows: 10000,  Columns: 784
Training :   (50000, 784) (50000,)
Validation:  (10000, 784) (10000,)
Test Set :   (10000, 784) (10000,)


In [2]:
def batch_generator(X, y, batch_size=64, 
                    shuffle=False, random_seed=None):
    
    idx = np.arange(y.shape[0])
    
    if shuffle:
        rng = np.random.RandomState(random_seed)
        rng.shuffle(idx)
        X = X[idx]
        y = y[idx]
    
    for i in range(0, X.shape[0], batch_size):
        yield (X[i:i+batch_size, :], y[i:i+batch_size])

In [3]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train - mean_vals)/std_val
X_valid_centered = X_valid - mean_vals
X_test_centered = (X_test - mean_vals)/std_val

del X_train, X_valid, X_test
del X_data, y_data

78.6661972213


In [4]:
import tensorflow as tf
import numpy as np


class ConvNN(object):
    def __init__(self, batchsize=64,
                 epochs=20, learning_rate=1e-4, 
                 dropout_rate=0.5,
                 shuffle=True, random_seed=None):
        np.random.seed(random_seed)
        self.batchsize = batchsize
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate
        self.shuffle = shuffle
                
        g = tf.Graph()
        with g.as_default():
            ## set random-seed:
            tf.set_random_seed(random_seed)
            
            ## build the network:
            self.build()

            ## initializer
            self.init_op = \
                tf.global_variables_initializer()

            ## saver
            self.saver = tf.train.Saver()
            
        ## create a session
        self.sess = tf.Session(graph=g)
                
    def build(self):
        
        ## Placeholders for X and y:
        tf_x = tf.placeholder(tf.float32, 
                              shape=[None, 784],
                              name='tf_x')
        tf_y = tf.placeholder(tf.int32, 
                              shape=[None],
                              name='tf_y')
        is_train = tf.placeholder(tf.bool, 
                              shape=(),
                              name='is_train')

        ## reshape x to a 4D tensor: 
        ##  [batchsize, width, height, 1]
        tf_x_image = tf.reshape(tf_x, shape=[-1,28,28,1],
                              name='input_x_2dimages')
        ## One-hot encoding:
        tf_y_onehot = tf.one_hot(indices=tf_y, depth=10,
                              dtype=tf.float32,
                              name='input_y_onehot')

        ## 1st layer: Conv_1
        h1 = tf.layers.conv2d(tf_x_image, 
                              kernel_size=(5,5), 
                              filters=32, 
                              activation=tf.nn.relu)
        ## MaxPooling
        h1_pool = tf.layers.max_pooling2d(h1, 
                              pool_size=(2,2), 
                              strides=(2,2))
        ## 2n layer: Conv_2
        h2 = tf.layers.conv2d(h1_pool, kernel_size=(5,5), 
                              filters=64, 
                              activation=tf.nn.relu)
        ## MaxPooling 
        h2_pool = tf.layers.max_pooling2d(h2, 
                              pool_size=(2,2), 
                              strides=(2,2))

        ## 3rd layer: Fully Connected
        input_shape = h2_pool.get_shape().as_list()
        n_input_units = np.prod(input_shape[1:])
        h2_pool_flat = tf.reshape(h2_pool, 
                              shape=[-1, n_input_units])
        h3 = tf.layers.dense(h2_pool_flat, 1024, 
                              activation=tf.nn.relu)

        ## Dropout
        h3_drop = tf.layers.dropout(h3, 
                              rate=self.dropout_rate,
                              training=is_train)
        
        ## 4th layer: Fully Connected (linear activation)
        h4 = tf.layers.dense(h3_drop, 10, 
                              activation=None)

        ## Prediction
        predictions = {
            'probabilities' : tf.nn.softmax(h4, 
                              name='probabilities'),
            'labels' : tf.cast(tf.argmax(h4, axis=1), 
                              tf.int32, name='labels')
        }
        
        ## Loss Function and Optimization
        cross_entropy_loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits=h4, labels=tf_y_onehot),
            name='cross_entropy_loss')
        
        ## Optimizer:
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        optimizer = optimizer.minimize(cross_entropy_loss,
                              name='train_op')

        ## Finding accuracy
        correct_predictions = tf.equal(
            predictions['labels'], 
            tf_y, name='correct_preds')
        
        accuracy = tf.reduce_mean(
            tf.cast(correct_predictions, tf.float32),
            name='accuracy')

    def save(self, epoch, path='./tflayers-model/'):
        if not os.path.isdir(path):
            os.makedirs(path)
        print('Saving model in {}'.format(path))
        self.saver.save(self.sess, 
                        os.path.join(path, 'model.ckpt'),
                        global_step=epoch)
        
    def load(self, epoch, path):
        print('Loading model from {}'.format(path))
        self.saver.restore(self.sess, 
             os.path.join(path, 'model.ckpt-{}'.format(epoch)))
        
    def train(self, training_set, 
              validation_set=None,
              initialize=True):
        ## initialize variables
        if initialize:
            self.sess.run(self.init_op)

        self.train_cost_ = []
        X_data = np.array(training_set[0])
        y_data = np.array(training_set[1])

        for epoch in range(1, self.epochs+1):
            batch_generator = \
                create_batch_generator(X_data, y_data, 
                              shuffle=self.shuffle)
            avg_loss = 0.0
            for i,(batch_x,batch_y) in \
                enumerate(batch_generator):
                feed = {'tf_x:0' : batch_x, 
                        'tf_y:0' : batch_y,
                        'is_train:0' : True} ## for dropout
                loss, _ = self.sess.run(
                        ['cross_entropy_loss:0', 'train_op'], 
                        feed_dict=feed)
                avg_loss += loss
                
            print('Epoch {}: Training Avg. Loss: '
                  '{:.3f}'.format(epoch, avg_loss), 
                  end='    ')
            if validation_set is not None:
                feed = {'tf_x:0' : batch_x, 
                        'tf_y:0' : batch_y,
                        'is_train:0' : False} ## for dropout
                valid_acc = self.sess.run('accuracy:0',
                        feed_dict=feed)
                print('Validatio Acc: {:.3f}'.format(valid_acc))
            else:
                print()
                    
    def predict(self, X_test, return_proba = False):
        feed = {'tf_x:0' : X_test,
                'is_train:0' : False} ## for dropout
        if return_proba:
            return self.sess.run('probabilities:0',
                                feed_dict=feed)
        else:
            return self.sess.run('labels:0',
                                feed_dict=feed)



In [5]:
cnn = ConvNN(random_seed=123)

In [6]:
cnn.train(training_set=(X_train_centered, y_train), 
          validation_set=(X_valid_centered, y_valid))

cnn.save(epoch=20)

Epoch 1: Training Avg. Loss: 181.439    Validatio Acc: 0.938
Epoch 2: Training Avg. Loss: 49.781    Validatio Acc: 0.962
Epoch 3: Training Avg. Loss: 34.668    Validatio Acc: 0.962
Epoch 4: Training Avg. Loss: 26.059    Validatio Acc: 0.975
Epoch 5: Training Avg. Loss: 21.463    Validatio Acc: 0.988
Epoch 6: Training Avg. Loss: 18.172    Validatio Acc: 0.988
Epoch 7: Training Avg. Loss: 15.444    Validatio Acc: 0.988
Epoch 8: Training Avg. Loss: 14.103    Validatio Acc: 1.000
Epoch 9: Training Avg. Loss: 12.264    Validatio Acc: 0.988
Epoch 10: Training Avg. Loss: 10.983    Validatio Acc: 1.000
Epoch 11: Training Avg. Loss: 10.399    Validatio Acc: 1.000
Epoch 12: Training Avg. Loss: 8.859    Validatio Acc: 1.000
Epoch 13: Training Avg. Loss: 7.595    Validatio Acc: 1.000
Epoch 14: Training Avg. Loss: 7.093    Validatio Acc: 1.000
Epoch 15: Training Avg. Loss: 6.310    Validatio Acc: 1.000
Epoch 16: Training Avg. Loss: 6.048    Validatio Acc: 1.000
Epoch 17: Training Avg. Loss: 5.373  

In [5]:
del cnn

cnn2 = ConvNN(random_seed=123)

cnn2.load(epoch=20, path='./tflayers-model/')

print(cnn2.predict(X_test_centered[:10,:]))

Loading model from ./tflayers-model/
INFO:tensorflow:Restoring parameters from ./tflayers-model/model.ckpt-20
[7 2 1 0 4 1 4 9 5 9]


In [6]:
preds = cnn2.predict(X_test_centered)

print('Test Accuracy: {:.2f}%'.format(100*
       np.sum(y_test == preds)/len(y_test)))

Test Accuracy: 99.27%
